## Create datasets for the Content-based Filter

This notebook builds the data we will use for creating our content based model. We'll collect the data via a collection of SQL queries from the publicly avialable Kurier.at dataset in BigQuery.
Kurier.at is an Austrian newsite. The goal of these labs is to recommend an article for a visitor to the site. In this lab we collect the data for training. In the subsequent notebook we train the recommender model. 

This notebook illustrates
* how to pull data from BigQuery table and write to local files
* how to make reproducible train and test splits 

In [1]:
import os
import tensorflow as tf
import numpy as np
from google.cloud import bigquery

PROJECT = 'qwiklabs-gcp-d053f1f13818fc74' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'qwiklabs-gcp-d053f1f13818fc74' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

# do not change these
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'

In [2]:
%%bash
gcloud  config  set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


We will use this helper funciton to write lists containing article ids, categories, and authors for each article in our database to local file. 

In [3]:
def write_list_to_disk(my_list, filename):
  with open(filename, 'w') as f:
    for item in my_list:
        line = "%s\n" % item
        f.write(str(line.encode('utf8')))

### Pull data from BigQuery

The cell below creates a local text file containing all the article ids (i.e. 'content ids') in the dataset. 

Have a look at the original dataset in [BigQuery](https://console.cloud.google.com/bigquery?p=cloud-training-demos&d=GA360_test&t=ga_sessions_sample). Then read through the query below and make sure you understand what it is doing. 

In [4]:
sql="""
#standardSQL
SELECT  
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id 
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY
  content_id
"""
content_ids_list = bigquery.Client().query(sql).to_dataframe()['content_id'].tolist()
write_list_to_disk(content_ids_list, "content_ids.txt")
print("Some sample content IDs {}".format(content_ids_list[:3]))
print("The total number of articles is {}".format(len(content_ids_list)))

Some sample content IDs [u'299922662', u'299826775', u'299437612']
The total number of articles is 15634


In the following cells, you will create a local file which contains a list of article categories and a list of article authors.

**Hint**: For the TODO below, modify the query above changing 'content_id' to the necessary field and changing index=10 
to the appropriate index.  
Refer back to the original dataset, use the `hits.customDimensions.index` 
field to find the correct index.

In [7]:
sql="""
#standardSQL
SELECT  
  (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category  
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY   
  category
"""
categories_list = bigquery.Client().query(sql).to_dataframe()['category'].tolist()
write_list_to_disk(categories_list, "categories.txt")
print(categories_list)

[u'Stars & Kultur', u'News', u'Lifestyle']


When creating the author list, we'll only use the first author information for each article.  

In [8]:
sql="""
#standardSQL
SELECT
  REGEXP_EXTRACT((SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)), r"^[^,]+")  AS first_author  
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY   
  first_author
"""
authors_list = bigquery.Client().query(sql).to_dataframe()['first_author'].tolist()
write_list_to_disk(authors_list, "authors.txt")
print("Some sample authors {}".format(authors_list[:10]))
print("The total number of authors is {}".format(len(authors_list)))

Some sample authors [u'Marlene Patsalidis', u'Yvonne Widler', u'Thomas  Trescher', u'Johanna Hager', u'Elisabeth Spitzer', u'Daniela Wahl', u'Stefan Berndl', u'Heinz Wagner', u'Brigitte Schokarth', u'Mathias Kainz']
The total number of authors is 385


### Create train and test sets.

In this section, we will create the train/test split of our data for training our model. Read through the query and complete the TODO at the bottom.  
Use the concatenated values for visitor id and content id to create a farm fingerprint, taking 90% of the data for the training set. 

In [11]:
sql="""
WITH site_history as (
  SELECT
      fullVisitorId as visitor_id,
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id,
      (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category, 
      (SELECT MAX(IF(index=6, value, NULL)) FROM UNNEST(hits.customDimensions)) AS title,
      (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) AS author_list,
      SPLIT(RPAD((SELECT MAX(IF(index=4, value, NULL)) FROM UNNEST(hits.customDimensions)), 7), '.') as year_month_array,
      LEAD(hits.customDimensions, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) as nextCustomDimensions
  FROM 
    `cloud-training-demos.GA360_test.ga_sessions_sample`,   
     UNNEST(hits) AS hits
   WHERE 
     # only include hits on pages
      hits.type = "PAGE"
      AND
      fullVisitorId IS NOT NULL
      AND
      hits.time != 0
      AND
      hits.time IS NOT NULL
      AND
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
)
SELECT
  visitor_id,
  content_id,
  category,
  REGEXP_REPLACE(title, r",", "") as title,
  REGEXP_EXTRACT(author_list, r"^[^,]+") as author,
  DATE_DIFF(DATE(CAST(year_month_array[OFFSET(0)] AS INT64), CAST(year_month_array[OFFSET(1)] AS INT64), 1), DATE(1970,1,1), MONTH) as months_since_epoch,
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) as next_content_id
FROM
  site_history
WHERE (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) IS NOT NULL AND
      MOD(ABS(FARM_FINGERPRINT(CONCAT(visitor_id, content_id))), 10) < 9
      #TODO: Use FARM_FINGERPRINT on the concatenated visitor_id and content_id to create a training set of approximately 90% of the data
"""

training_set_df = bigquery.Client().query(sql).to_dataframe()
training_set_df.to_csv('training_set.csv', header=False, index=False, encoding='utf-8')
training_set_df.head()

,visitor_id,content_id,category,title,author,months_since_epoch,next_content_id
0,1000593816586876859,230814320,Stars & Kultur,Kritik an Meghan Markle immer lauter,Elisabeth Spitzer,562,299837992
1,1001769331926555188,299836255,News,Blümel Kneissl &Co.: Das sind die Fixstarter,None,574,299826767
2,1001769331926555188,299826767,Lifestyle,Titanic-Regisseur: Darum musste Jack sterben,Elisabeth Mittendorfer,574,299921761
3,1001769331926555188,299912085,News,Erster ÖBB-Containerzug nach China unterwegs,Stefan Hofer,574,299836841
4,1001769331926555188,299836841,News,ÖVP will Studiengebühren FPÖ in Verhandlungen ...,Raffaela Lindorfer,574,299915880


Repeat the query as above but change outcome of the farm fingerprint hash to collect the remaining 10% of the data for the test set.

In [13]:
sql="""
WITH site_history as (
  SELECT
      fullVisitorId as visitor_id,
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id,
      (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category, 
      (SELECT MAX(IF(index=6, value, NULL)) FROM UNNEST(hits.customDimensions)) AS title,
      (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) AS author_list,
      SPLIT(RPAD((SELECT MAX(IF(index=4, value, NULL)) FROM UNNEST(hits.customDimensions)), 7), '.') as year_month_array,
      LEAD(hits.customDimensions, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) as nextCustomDimensions
  FROM 
    `cloud-training-demos.GA360_test.ga_sessions_sample`,   
     UNNEST(hits) AS hits
   WHERE 
     # only include hits on pages
      hits.type = "PAGE"
      AND
      fullVisitorId IS NOT NULL
      AND
      hits.time != 0
      AND
      hits.time IS NOT NULL
      AND
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
)
SELECT
  visitor_id,
  content_id,
  category,
  REGEXP_REPLACE(title, r",", "") as title,
  REGEXP_EXTRACT(author_list, r"^[^,]+") as author,
  DATE_DIFF(DATE(CAST(year_month_array[OFFSET(0)] AS INT64), CAST(year_month_array[OFFSET(1)] AS INT64), 1), DATE(1970,1,1), MONTH) as months_since_epoch,
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) as next_content_id
FROM
  site_history
WHERE (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) IS NOT NULL AND
      #TODO: Modify the FARM_FINGERPRINT you used in the previous cell to create a test set of approximately 10% of the data
      MOD(ABS(FARM_FINGERPRINT(CONCAT(visitor_id, content_id))), 10) >= 9
"""

test_set_df = bigquery.Client().query(sql).to_dataframe()
test_set_df.to_csv('test_set.csv', header=False, index=False, encoding='utf-8')
test_set_df.head()

,visitor_id,content_id,category,title,author,months_since_epoch,next_content_id
0,1000196974485173657,299925700,Lifestyle,Nach Tod von Vater: Tochter bekommt jedes Jahr...,Marlene Patsalidis,574,299972194
1,1000196974485173657,299972194,News,LIVE: Spielstand bei Sturm - Admira,Mathias Kainz,574,299816215
2,1000196974485173657,299972194,News,Dominante Grazer nehmen Admira auseinander,Mathias Kainz,574,299410466
3,1007505561418545529,299407839,Stars & Kultur,Trump: 165 Millionen Dollar für 5 Tage Urlaub,Elisabeth Spitzer,574,299816215
4,1017855659516706306,298846345,Stars & Kultur,Meghan Markle: Lottogewinn veränderte ihr Leben,Elisabeth Spitzer,574,299814775


Let's have a look at the two csv files we just created containing the training and test set. We'll also do a line count of both files to confirm that we have achieved an approximate 90/10 train/test split.  
In the next notebook, **Content Based Filtering** we will build a model to recommend an article given information about the current article being read, such as the category, title, author, and publish date. 

In [14]:
%%bash
wc -l *_set.csv

   25599 test_set.csv
  232308 training_set.csv
  257907 total


In [15]:
!head *_set.csv

==> test_set.csv <==
1000196974485173657,299925700,Lifestyle,Nach Tod von Vater: Tochter bekommt jedes Jahr Blumen,Marlene Patsalidis,574,299972194
1000196974485173657,299972194,News,LIVE: Spielstand bei Sturm - Admira,Mathias Kainz,574,299816215
1000196974485173657,299972194,News,Dominante Grazer nehmen Admira auseinander,Mathias Kainz,574,299410466
1007505561418545529,299407839,Stars & Kultur,Trump: 165 Millionen Dollar für 5 Tage Urlaub,Elisabeth Spitzer,574,299816215
1017855659516706306,298846345,Stars & Kultur,Meghan Markle: Lottogewinn veränderte ihr Leben,Elisabeth Spitzer,574,299814775
1020191185375536307,299827911,News,"""Vulkanausbrüche sind normal""",Michaela Reibenwein,574,187077794
1021039862358799317,299911496,News,Papst auf Myanmar-Besuch: Franziskus trifft Aung San Suu Kyi,,574,299912041
1032965470337888105,299826775,Lifestyle,Auf Bank ausgeruht: Pensionist muss Strafe zahlen,Marlene Patsalidis,574,299918253
1034151839179551865,297547655,Stars & Kultur,"Ozzy Osbourne sp